# Week 11: MLOps Production Features

Production-grade ML operations including feature stores, model registry, automated retraining, and drift detection.

**Learning Objectives**:
- Implement feature store for consistent feature engineering
- Build model registry with versioning
- Create automated retraining pipelines
- Detect and alert on data/model drift

**Production Impact**: These components are critical for maintaining ML systems in production.

In [ ]:
import numpy as np
import json
from datetime import datetime, timedelta
from pathlib import Path
import pickle
from typing import Dict, List, Any

print("✅ Imports complete")

## 1. Feature Store Implementation

A feature store provides:
- **Consistency**: Same features for training and serving
- **Reusability**: Share features across models
- **Freshness**: Automated feature updates
- **Governance**: Track feature lineage

In [ ]:
class FeatureStore:
    """
    Simple feature store for ML features.
    
    In production, use Feast, Tecton, or Hopsworks.
    """
    
    def __init__(self, store_path: str = "./feature_store"):
        self.store_path = Path(store_path)
        self.store_path.mkdir(exist_ok=True)
        
        self.features_db = {}  # In-memory cache
        self.metadata_path = self.store_path / "metadata.json"
        self._load_metadata()
    
    def _load_metadata(self):
        """Load feature metadata."""
        if self.metadata_path.exists():
            with open(self.metadata_path, 'r') as f:
                self.metadata = json.load(f)
        else:
            self.metadata = {}
    
    def register_feature(self, feature_name: str, feature_type: str, 
                        description: str, tags: List[str] = None):
        """Register a new feature."""
        self.metadata[feature_name] = {
            'type': feature_type,
            'description': description,
            'tags': tags or [],
            'created_at': datetime.now().isoformat(),
            'updated_at': datetime.now().isoformat()
        }
        self._save_metadata()
    
    def write_features(self, entity_id: str, features: Dict[str, Any]):
        """Write features for an entity (e.g., user_id, transaction_id)."""
        feature_file = self.store_path / f"{entity_id}.pkl"
        
        with open(feature_file, 'wb') as f:
            pickle.dump({
                'features': features,
                'timestamp': datetime.now().isoformat()
            }, f)
        
        # Update metadata
        for feature_name in features.keys():
            if feature_name in self.metadata:
                self.metadata[feature_name]['updated_at'] = datetime.now().isoformat()
        self._save_metadata()
    
    def read_features(self, entity_id: str, feature_names: List[str] = None) -> Dict:
        """Read features for an entity."""
        feature_file = self.store_path / f"{entity_id}.pkl"
        
        if not feature_file.exists():
            raise ValueError(f"No features found for entity {entity_id}")
        
        with open(feature_file, 'rb') as f:
            data = pickle.load(f)
        
        features = data['features']
        
        if feature_names:
            features = {k: v for k, v in features.items() if k in feature_names}
        
        return features
    
    def get_feature_metadata(self, feature_name: str) -> Dict:
        """Get metadata for a feature."""
        return self.metadata.get(feature_name, {})
    
    def _save_metadata(self):
        """Save metadata to disk."""
        with open(self.metadata_path, 'w') as f:
            json.dump(self.metadata, f, indent=2)


# Example usage
store = FeatureStore()

# Register features
store.register_feature(
    'user_age',
    'int',
    'User age in years',
    tags=['demographic', 'user']
)

store.register_feature(
    'purchase_count_30d',
    'int',
    'Number of purchases in last 30 days',
    tags=['behavioral', 'aggregation']
)

# Write features
store.write_features('user_12345', {
    'user_age': 28,
    'purchase_count_30d': 5,
    'avg_basket_size': 45.50
})

# Read features
features = store.read_features('user_12345', ['user_age', 'purchase_count_30d'])
print(f"Features: {features}")

print("\n✅ Feature store implementation complete")

## 2. Model Registry

Track all trained models with versioning, metrics, and metadata.

In [ ]:
class ModelRegistry:
    """
    Model registry for tracking trained models.
    
    In production, use MLflow Model Registry or similar.
    """
    
    def __init__(self, registry_path: str = "./model_registry"):
        self.registry_path = Path(registry_path)
        self.registry_path.mkdir(exist_ok=True)
        
        self.models_path = self.registry_path / "models"
        self.models_path.mkdir(exist_ok=True)
        
        self.catalog_path = self.registry_path / "catalog.json"
        self._load_catalog()
    
    def _load_catalog(self):
        """Load model catalog."""
        if self.catalog_path.exists():
            with open(self.catalog_path, 'r') as f:
                self.catalog = json.load(f)
        else:
            self.catalog = {}
    
    def register_model(self, model_name: str, model_obj: Any, 
                      metrics: Dict[str, float], metadata: Dict = None):
        """Register a trained model."""
        # Generate version number
        if model_name not in self.catalog:
            self.catalog[model_name] = {'versions': []}
        
        version = len(self.catalog[model_name]['versions']) + 1
        version_str = f"v{version}"
        
        # Save model
        model_path = self.models_path / f"{model_name}_{version_str}.pkl"
        with open(model_path, 'wb') as f:
            pickle.dump(model_obj, f)
        
        # Record in catalog
        version_info = {
            'version': version_str,
            'created_at': datetime.now().isoformat(),
            'metrics': metrics,
            'metadata': metadata or {},
            'model_path': str(model_path),
            'status': 'staging'  # staging, production, archived
        }
        
        self.catalog[model_name]['versions'].append(version_info)
        self._save_catalog()
        
        return version_str
    
    def promote_to_production(self, model_name: str, version: str):
        """Promote a model version to production."""
        # Demote current production model
        for v in self.catalog[model_name]['versions']:
            if v['status'] == 'production':
                v['status'] = 'archived'
        
        # Promote new version
        for v in self.catalog[model_name]['versions']:
            if v['version'] == version:
                v['status'] = 'production'
                v['promoted_at'] = datetime.now().isoformat()
        
        self._save_catalog()
    
    def get_model(self, model_name: str, version: str = 'production'):
        """Load a model by name and version."""
        if version == 'production':
            # Find production version
            for v in self.catalog[model_name]['versions']:
                if v['status'] == 'production':
                    model_path = v['model_path']
                    break
        else:
            # Find specific version
            for v in self.catalog[model_name]['versions']:
                if v['version'] == version:
                    model_path = v['model_path']
                    break
        
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
        
        return model
    
    def list_models(self) -> List[str]:
        """List all registered models."""
        return list(self.catalog.keys())
    
    def get_model_info(self, model_name: str) -> Dict:
        """Get info about a model."""
        return self.catalog.get(model_name, {})
    
    def _save_catalog(self):
        """Save catalog to disk."""
        with open(self.catalog_path, 'w') as f:
            json.dump(self.catalog, f, indent=2)


# Example usage
registry = ModelRegistry()

# Train a model (mock)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

# Register model
version = registry.register_model(
    'churn_classifier',
    model,
    metrics={'accuracy': 0.87, 'f1': 0.82},
    metadata={'dataset': 'customers_2024_01', 'features': 47}
)

print(f"Registered model version: {version}")

# Promote to production
registry.promote_to_production('churn_classifier', version)
print(f"Promoted {version} to production")

# Load production model
prod_model = registry.get_model('churn_classifier', 'production')
print(f"Loaded production model: {prod_model}")

print("\n✅ Model registry implementation complete")

## 3. Automated Retraining Pipeline

Automatically retrain models when:
- Performance drops below threshold
- On schedule (weekly, monthly)
- New data available

In [ ]:
class RetrainingPipeline:
    """
    Automated model retraining pipeline.
    
    Monitors performance and triggers retraining when needed.
    """
    
    def __init__(self, model_registry: ModelRegistry):
        self.registry = model_registry
        self.performance_log = []
    
    def should_retrain(self, current_metrics: Dict[str, float], 
                      threshold: float = 0.05) -> bool:
        """
        Determine if model should be retrained.
        
        Args:
            current_metrics: Current production metrics
            threshold: Performance degradation threshold
        
        Returns:
            True if retraining recommended
        """
        # Log current metrics
        self.performance_log.append({
            'timestamp': datetime.now().isoformat(),
            'metrics': current_metrics
        })
        
        if len(self.performance_log) < 2:
            return False
        
        # Compare to baseline (first week performance)
        baseline = self.performance_log[0]['metrics']
        
        for metric_name, current_value in current_metrics.items():
            baseline_value = baseline.get(metric_name, current_value)
            degradation = (baseline_value - current_value) / baseline_value
            
            if degradation > threshold:
                print(f"⚠️ Performance degraded: {metric_name} dropped by {degradation:.2%}")
                return True
        
        return False
    
    def retrain_model(self, model_name: str, X_train, y_train):
        """
        Retrain model with new data.
        
        In production, this would:
        1. Pull latest data from warehouse
        2. Generate features from feature store
        3. Train new model
        4. Evaluate on holdout set
        5. Register in model registry
        6. Run A/B test before promoting
        """
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.metrics import accuracy_score, f1_score
        from sklearn.model_selection import train_test_split
        
        # Split for validation
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42
        )
        
        # Train new model
        print(f"🔄 Retraining {model_name}...")
        new_model = RandomForestClassifier(n_estimators=100, random_state=42)
        new_model.fit(X_tr, y_tr)
        
        # Evaluate
        y_pred = new_model.predict(X_val)
        metrics = {
            'accuracy': accuracy_score(y_val, y_pred),
            'f1': f1_score(y_val, y_pred, average='weighted')
        }
        
        print(f"✅ New model metrics: {metrics}")
        
        # Register new version
        version = self.registry.register_model(
            model_name,
            new_model,
            metrics,
            metadata={'retrained_at': datetime.now().isoformat()}
        )
        
        return version, metrics


# Example usage
pipeline = RetrainingPipeline(registry)

# Simulate production monitoring
week1_metrics = {'accuracy': 0.87, 'f1': 0.82}
week2_metrics = {'accuracy':  0.85, 'f1': 0.80}
week3_metrics = {'accuracy': 0.80, 'f1': 0.75}  # Drops >5%

pipeline.should_retrain(week1_metrics)
pipeline.should_retrain(week2_metrics)

if pipeline.should_retrain(week3_metrics, threshold=0.05):
    print("\n🚨 Triggering retraining...")
    # Would retrain here with real data
    # version, metrics = pipeline.retrain_model('churn_classifier', X_new, y_new)

print("\n✅ Retraining pipeline implementation complete")

## 4. Data Drift Detection

Detect when input data distribution changes (concept drift).

In [ ]:
from scipy import stats

class DriftDetector:
    """
    Detect data/concept drift using statistical tests.
    
    Methods:
    - Kolmogorov-Smirnov test (continuous features)
    - Chi-square test (categorical features)
    - Population Stability Index (PSI)
    """
    
    def __init__(self, reference_data: np.ndarray):
        """Store reference (training) data distribution."""
        self.reference_data = reference_data
    
    def ks_test(self, production_data: np.ndarray, threshold: float = 0.05) -> Dict:
        """
        Kolmogorov-Smirnov test for continuous features.
        
        H0: Production data comes from same distribution as reference
        Reject H0 if p-value < threshold (drift detected)
        """
        results = {}
        
        for i in range(self.reference_data.shape[1]):
            ref_feature = self.reference_data[:, i]
            prod_feature = production_data[:, i]
            
            statistic, p_value = stats.ks_2samp(ref_feature, prod_feature)
            
            results[f'feature_{i}'] = {
                'statistic': statistic,
                'p_value': p_value,
                'drift_detected': p_value < threshold
            }
        
        return results
    
    def psi(self, production_data: np.ndarray, num_bins: int = 10) -> Dict:
        """
        Population Stability Index.
        
        PSI < 0.1: No significant change
        0.1 < PSI < 0.2: Small change
        PSI > 0.2: Significant drift
        """
        results = {}
        
        for i in range(self.reference_data.shape[1]):
            ref_feature = self.reference_data[:, i]
            prod_feature = production_data[:, i]
            
            # Create bins
            _, bins = np.histogram(ref_feature, bins=num_bins)
            
            # Bin counts
            ref_counts, _ = np.histogram(ref_feature, bins=bins)
            prod_counts, _ = np.histogram(prod_feature, bins=bins)
            
            # Convert to proportions
            ref_props = (ref_counts + 1) / (len(ref_feature) + num_bins)  # +1 smoothing
            prod_props = (prod_counts + 1) / (len(prod_feature) + num_bins)
            
            # Calculate PSI
            psi_value = np.sum((prod_props - ref_props) * np.log(prod_props / ref_props))
            
            results[f'feature_{i}'] = {
                'psi': psi_value,
                'drift_severity': 'no_change' if psi_value < 0.1 else ('small' if psi_value < 0.2 else 'significant')
            }
        
        return results
    
    def detect_drift(self, production_data: np.ndarray) -> Dict:
        """Run all drift detection methods."""
        return {
            'ks_test': self.ks_test(production_data),
            'psi': self.psi(production_data),
            'timestamp': datetime.now().isoformat()
        }


# Example usage
# Reference data (training)
X_ref = np.random.randn(1000, 5)  # 1000 samples, 5 features

# Production data (no drift)
X_prod_no_drift = np.random.randn(500, 5)

# Production data (with drift on feature 0)
X_prod_drift = np.random.randn(500, 5)
X_prod_drift[:, 0] = X_prod_drift[:, 0] + 2.0  # Shift mean by 2

# Detect drift
detector = DriftDetector(X_ref)

print("Testing data with NO drift:")
results_no_drift = detector.detect_drift(X_prod_no_drift)
print(f"Feature 0 PSI: {results_no_drift['psi']['feature_0']['psi']:.4f}")
print(f"Feature 0 KS p-value: {results_no_drift['ks_test']['feature_0']['p_value']:.4f}")

print("\nTesting data WITH drift:")
results_drift = detector.detect_drift(X_prod_drift)
print(f"Feature 0 PSI: {results_drift['psi']['feature_0']['psi']:.4f} ({results_drift['psi']['feature_0']['drift_severity']})")
print(f"Feature 0 KS p-value: {results_drift['ks_test']['feature_0']['p_value']:.6f}")
print(f"Drift detected: {results_drift['ks_test']['feature_0']['drift_detected']}")

print("\n✅ Drift detection implementation complete")

## 5. Complete MLOps Workflow

Putting it all together: feature store → train → register → monitor → retrain

In [ ]:
def mlops_workflow_example():
    """
    Complete MLOps workflow demonstration.
    
    Steps:
    1. Feature engineering → Feature Store
    2. Model training
    3. Model registration
    4. Deployment to production
    5. Monitoring (drift + performance)
    6. Automated retraining when needed
    """
    print("🚀 MLOps Workflow Demo\n")
    
    # 1. Feature Store
    print("Step 1: Feature Engineering")
    feature_store = FeatureStore()
    
    # Would compute features for all users
    for user_id in ['user_1', 'user_2', 'user_3']:
        features = {
            'age': np.random.randint(18, 65),
            'purchase_count': np.random.randint(0, 50),
            'avg_spend': np.random.uniform(10, 200)
        }
        feature_store.write_features(user_id, features)
    
    print("✓ Features stored\n")
    
    # 2. Model Training
    print("Step 2: Model Training")
    # Would pull features from store
    # X_train = get_features_from_store(user_ids)
    print("✓ Model trained\n")
    
    # 3. Model Registry
    print("Step 3: Model Registration")
    model_registry = ModelRegistry()
    # version = model_registry.register_model(...)
    print("✓ Model registeredno\n")
    
    # 4. Production Deployment
    print("Step 4: Deploy to Production")
    # model_registry.promote_to_production(model_name, version)
    print("✓ Model in production\n")
    
    # 5. Monitoring
    print("Step 5: Monitoring")
    # drift_detector = DriftDetector(X_train)
    # drift_results = drift_detector.detect_drift(X_prod)
    print("✓ Monitoring active\n")
    
    # 6. Automated Retraining
    print("Step 6: Auto-Retraining (if needed)")
    # retraining_pipeline = RetrainingPipeline(model_registry)
    # if retraining_pipeline.should_retrain(current_metrics):
    #     retraining_pipeline.retrain_model(...)
    print("✓ Retraining pipeline ready\n")
    
    print("✅ Complete MLOps workflow demonstrated!")

mlops_workflow_example()

## Key Takeaways

### Production Best Practices

1. **Feature Store**: Ensures consistency between training and serving
2. **Model Registry**: Version control for ML models
3. **Automated Retraining**: Keep models fresh without manual intervention
4. **Drift Detection**: Alert when data distribution changes

### Tools in Production

- **Feature Stores**: Feast, Tecton, Hopsworks
- **Model Registry**: MLflow, Weights & Biases
- **Orchestration**: Airflow, Prefect, Dagster
- **Monitoring**: Evidently AI, WhyLabs, Arize

### Next Steps

1. Integrate with existing production API
2. Add alerting (PagerDuty, Slack)
3. Create dashboards (Grafana)
4. Set up CI/CD for model deployments

---

**This notebook provides a foundation for production MLOps!**
